In [ ]:
import mne
import os
import psutil
import gc

import utils.config as config
from utils.helpers import perform_ica_cleaning
from utils.helpers import save_epochs
from utils.config import DATASETS

%matplotlib qt

### Defining constants and preparing stuff ###

bids_root = os.path.join(DATASETS["Jin2019"].path, "study1/raw/eeg/")
path_epochs = os.path.join(config.EPOCHS_PATH, "external_task/jin2019/")

# EEG settings
subjects = DATASETS["Jin2019"].subjects
sessions = DATASETS["Jin2019"].sessions

process = psutil.Process()

### Defining functions ###
   
def load_epochs(subject, session, bids_root=path_epochs):
    """
    Load MNE epochs from a file with structured filename.

    Parameters:
    -----------
    subject : str or int
        Subject identifier.
    session : str or int
        Session identifier.
    bids_root : str, optional
        Directory where the epoch files are stored. Defaults to "./epochs".

    Returns:
    --------
    mne.Epochs
        The loaded epochs object.
    """
    # Construct filename
    filename = f"sub-{subject}_ses-{session}_epo.fif"
    file_path = os.path.join(bids_root, filename)
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None

    # Load the epochs
    epochs = mne.read_epochs(file_path, preload=True)

    print(f"Loaded epochs from: {file_path}")
    return epochs

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


**ICA cleaning the epochs**

In [ ]:
# Load in the epochs and do ICA
for subject in subjects:
    for session in sessions:
        epochs = load_epochs(subject, session, path_epochs)

        # Skip if epochs are not loaded
        if epochs is None:
            continue

        # Perform ICA
        cleaned_epochs, ica = perform_ica_cleaning(epochs, subject)

        # Save the epochs
        save_epochs(cleaned_epochs, path_epochs, subject=subject, session=session, suffix="ica_cleaned")

        del epochs
        gc.collect()
        print(f"Memory after cleanup: {process.memory_info().rss / 1e6:.2f} MB")

File not found: ./epochs/external_task (another copy)/sub-1_ses-1_epo.fif
Reading /home/sivert/Documents/Specialization_project/epochs/external_task (another copy)/sub-1_ses-2_epo.fif ...
    Found the data of interest:
        t =   -5000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
328 matching events found
No baseline correction applied
0 projection items activated
Loaded epochs from: ./epochs/external_task (another copy)/sub-1_ses-2_epo.fif
Fitting ICA...
Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 20 components
Computing Infomax ICA
Fitting ICA took 18.9s.
ICA fitting complete.
Displaying ICA components. Use the interactive GUI to select components for exclusion.
Displaying ICA sources. Use this to examine the time-series for artifact-related components.
Not setting metadata
328 matching events found
No baseline correction applied
0 projection items activated
Applying ICA to remove